# Notebook principal pour la classe AvisCritique
Ce notebook sert à développer, tester et exporter la classe AvisCritique pour le projet LMELP.

In [ ]:
# |default_exp mongo_avis_critique
from mongo import BaseEntity, get_collection
from typing import TypeVar
from date_utils import format_date, DATE_FORMAT
import datetime
from config import get_DB_VARS

## Définition de la classe AvisCritique

In [ ]:
# |export
__all__ = ["T", "AvisCritique", "get_collection"]

In [ ]:
# |export
from mongo import BaseEntity


class AvisCritique(BaseEntity):
    collection: str = "avis_critiques"

    def __init__(
        self,
        episode_id,
        entity_type,
        entity_name,
        summary_text=None,
        created_at=None,
        updated_at=None,
    ):
        super().__init__(episode_id, self.collection)
        self.episode_id = episode_id
        self.entity_type = entity_type
        self.entity_name = entity_name
        self.summary_text = summary_text
        self.created_at = created_at or datetime.datetime.now()
        self.updated_at = updated_at or datetime.datetime.now()

    @property
    def nom(self):
        return f"{self.entity_type}_{self.entity_name}_{self.episode_id}"

    @classmethod
    def from_oid(cls, oid):
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        coll = get_collection(DB_HOST, DB_NAME, cls.collection)
        doc = coll.find_one({"_id": oid})
        if not doc:
            return None
        return cls(
            episode_id=doc.get("episode_id"),
            entity_type=doc.get("entity_type"),
            entity_name=doc.get("entity_name"),
            summary_text=doc.get("summary_text"),
            created_at=doc.get("created_at"),
            updated_at=doc.get("updated_at"),
        )

    @classmethod
    def find_by_episode_and_entity(cls, episode_id, entity_type, entity_name):
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        coll = get_collection(DB_HOST, DB_NAME, cls.collection)
        docs = coll.find(
            {
                "episode_id": episode_id,
                "entity_type": entity_type,
                "entity_name": entity_name,
            }
        )
        return [
            cls(
                episode_id=doc.get("episode_id"),
                entity_type=doc.get("entity_type"),
                entity_name=doc.get("entity_name"),
                summary_text=doc.get("summary_text"),
                created_at=doc.get("created_at"),
                updated_at=doc.get("updated_at"),
            )
            for doc in docs
        ]

    @classmethod
    def find_by_episode_id(cls, episode_id):
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        coll = get_collection(DB_HOST, DB_NAME, cls.collection)
        docs = coll.find({"episode_id": episode_id})
        return [
            cls(
                episode_id=doc.get("episode_id"),
                entity_type=doc.get("entity_type"),
                entity_name=doc.get("entity_name"),
                summary_text=doc.get("summary_text"),
                created_at=doc.get("created_at"),
                updated_at=doc.get("updated_at"),
            )
            for doc in docs
        ]

    def __str__(self):
        if self.summary_text:
            return f"AvisCritique({self.nom}): {self.summary_text}"
        return f"AvisCritique({self.nom})"

## Tests unitaires pour AvisCritique

In [ ]:
def test_initialisation():
    avis = AvisCritique(
        "507f1f77bcf86cd799439011",
        "Les Misérables",
        datetime.date(2023, 1, 15),
        "Résumé...",
        None,
        None,
    )
    assert avis.episode_title == "Les Misérables"
    assert avis.episode_date == format_date(datetime.date(2023, 1, 15), DATE_FORMAT)

## Export du module avec nbdev

In [ ]:
from nbdev.export import nb_export

nb_export("py mongo helper avis_critiques.ipynb", ".")